In [1]:
### RUN EVERY TIME: ECONOMIC INDICES RELEASES HISTORY EXTRACTING

In [2]:
### RUN EVERY TIME: INITIALIZATION

import pandas as pd
import numpy as np
from datetime import date, datetime
from math import floor, ceil
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.x13 import x13_arima_analysis
import os

%load_ext line_profiler

In [3]:
### RUN EVERY TIME: SEASONAL ADJUSTMENT MODULE PATHS SET UP

%env X13PATH = C:\Users\ighar\AppData\Roaming\jupyter\x13as
%env X12PATH = C:\Users\ighar\AppData\Roaming\jupyter\x13as
#%env

env: X13PATH=C:\Users\ighar\AppData\Roaming\jupyter\x13as
env: X12PATH=C:\Users\ighar\AppData\Roaming\jupyter\x13as


In [4]:
## VERSION CONTROL

from platform import python_version
print('pandas version: ', pd.__version__)
print('python version: ', python_version())

pandas version:  0.25.3
python version:  3.7.4


In [11]:
### RUN EVERY TIME: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
list_na_excel_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null',
                        '#N/A Requesting Data...', '#N/A Invalid Security', '#N/A Field Not Applicable']
### Raw data path and sheets:
str_path_bb_idx_source = 'Data_Files/Source_Files/Bloomberg_Eco_Indices.xlsx'
str_us_sheet = 'US Eco Const'
str_all_sheet = 'All Eco Const'
### Flags data path and sheets:
str_path_bb_idx_flags = 'Data_Files/Source_Files/Bloomberg_Eco_Flags_Extended.xlsx'
str_flag_sheet = 'Bloomberg Description'
### Source data constants:
int_idx_cols = 12
### HDF file with converted source data:
str_path_bb_idx_hdf = 'Data_Files/Source_Files/Bloomberg_Eco_Indices.h5'
str_key_flags = 'flags_exported' ### Acadian flags list
str_key_exported = 'all_idx_exported' ### Raw export with only replacing zero dates and after 2021-01-01 dates with np.NaN
str_key_raw_filled = 'all_idx_raw_filled' ### Raw export with initial dates, dates gaps, absent date coluns filled
str_key_raw_history = 'raw_history' ### Export with all the corrections and fillings (restructured to [Index_Name -> Data_Date -> Observation_Date] | Index_Value series)
str_key_bday_history = 'bday_history' ### Raw history vector with observation dates moved to nearest future business dates
str_key_num_history = 'num_history' ### Bday history vector with observation dates changed to their date numbers (for future matrix cube saving as hdf file)
str_key_from_date = 'idx_from_date' ### Series to get date numbers from dates
str_key_to_date = 'idx_to_date' ### Series to get dates from date numbers
### HDF file with matrices:
str_path_bb_matrix_hdf = 'Data_Files/Source_Files/Matrix_Eco_Indices.h5'
str_key_matrix_z = 'matrix_cube_z_scored'
### HDF file with diagonals:
str_path_bb_diag_hdf = 'Data_Files/Source_Files/Diag_Eco_Indices.h5'
str_key_diag_daily_raw = 'matrix_diagonal_raw'
str_key_diag_daily_z = 'matrix_diagonal_z'
### Observation axis range:
datetime_start = datetime(1984, 12, 31) # Start date for efficacy measures
date_start = datetime_start.date()
datetime_end = datetime(2020, 8, 31) # End date for efficacy measures
date_end = datetime_end.date()
idx_date_range = pd.date_range(date_start, date_end, freq = 'B')
datetime_basis = datetime(1993, 12, 31) # End date for efficacy measures
date_basis = datetime_basis.date()
### Gaps filling options:
int_revision_shift = 1
int_final_shift = 2
int_first_mean_length = 12
dict_final_only_lag = {}
dict_final_only_lag['Quarterly'] = 90 // 2
dict_final_only_lag['Monthly'] = 30 // 2
dict_final_only_lag['Other'] = 7 // 2
### Cumprod shifts for monthly data frequency:
dict_cumprod_step = {}
dict_cumprod_step['MoM%'] = 1
dict_cumprod_step['QoQ%'] = 3
dict_cumprod_step['YoY%'] = 12
### Stock-like series shifts for YoY transformation:
dict_yoy_shift = {}
dict_yoy_shift['Monthly'] = 12
dict_yoy_shift['Quarterly'] = 4
dict_yoy_shift['Other'] = 52
### Group tickers rebasing options:
int_not_to_rebase_term = 7 ### Term in years for min ticker data date when we do not need to rebase it with basis group ticker
int_not_to_rebase_diff = 2 ### Minimal difference in years between basis ticker and other group ticker min date when we need to rebase group ticker
### Z-scoring options:
int_winsorize_bound = 4
flo_winsorize_tolerance = 0.0001
int_winsorize_steps_limit = 5
### Diagonal options:
int_min_years = 3
date_diag_start = datetime(1994, 1, 1)

In [ ]:
### RUN TO RE-EXPORT DATA: SOURCE FILE EXPORT

### Reading excel file:
df_all_idx_raw = pd.read_excel(io = str_path_bb_idx_source, sheet_name = str_all_sheet, skiprows = [0], index_col = None, header = None, parse_dates = True,
                               na_values = list_na_excel_values, keep_default_na = False)
### List of dataframes for each eco index initializing:
list_idx_raw = []
### Extracting and converting each eco index data block seperately to proper form:
for int_iter_idx_col in range(len(df_all_idx_raw.columns) // int_idx_cols):
    ### Extracting raw data:
    df_iter_idx_raw = df_all_idx_raw.iloc[All, (int_iter_idx_col * int_idx_cols) : ((int_iter_idx_col + 1) * int_idx_cols) - 1]
    ### Assigning colum names:
    df_iter_idx_raw.columns = df_iter_idx_raw.iloc[1]
    df_iter_idx_raw.columns.name = ''    
    ### Dropping empty rows:
    df_iter_idx_raw.dropna(how = 'all', inplace = True)
    ### Extracting eco index name:
    df_iter_idx_raw['Index_Name'] = df_iter_idx_raw.iloc[0, 0]
    ### Dropping identification rows (no longer needed):
    df_iter_idx_raw.drop([0, 1], axis = 0, inplace = True)
    ### Setting multiindex:
    df_iter_idx_raw.set_index(['Date', 'Index_Name'], drop = True, append = False, inplace = True)
    ### ADding dataframe to list:
    list_idx_raw.append(df_iter_idx_raw)
### Exported data aggregating:
df_all_idx_exported = pd.concat(list_idx_raw, axis = 0).swaplevel()
### Exported data saving:
df_all_idx_exported.index.names = ['Index_Name', 'Data_Date']
### Types converting for numerical data:
df_all_idx_exported = df_all_idx_exported.astype({'PX_LAST': 'float',
                                                  'ACTUAL_RELEASE': 'float',
                                                  'FIRST_REVISION': 'float',
                                                  'BN_SURVEY_NUMBER_OBSERVATIONS': 'float',
                                                  'BN_SURVEY_MEDIAN': 'float',
                                                  'BN_SURVEY_AVERAGE': 'float',
                                                  'FORECAST_STANDARD_DEVIATION': 'float',
                                                 })
### Zero dates dropping:
df_all_idx_exported.loc[df_all_idx_exported['ECO_RELEASE_DT'] == 0, ['ECO_RELEASE_DT']] = np.NaN
df_all_idx_exported.loc[df_all_idx_exported['FIRST_REVISION_DATE'] == 0, ['FIRST_REVISION_DATE']] = np.NaN
df_all_idx_exported.loc[df_all_idx_exported['ECO_FUTURE_RELEASE_DATE'] == 0, ['ECO_FUTURE_RELEASE_DATE']] = np.NaN
### Incorrect dates droppping:
df_all_idx_exported.loc[df_all_idx_exported['ECO_RELEASE_DT'] > 20210000, 'ECO_RELEASE_DT'] = np.NaN
df_all_idx_exported.loc[df_all_idx_exported['FIRST_REVISION_DATE'] > 20210000, 'FIRST_REVISION_DATE'] = np.NaN
df_all_idx_exported.loc[df_all_idx_exported['ECO_FUTURE_RELEASE_DATE'] > 20210000, 'ECO_FUTURE_RELEASE_DATE'] = np.NaN
### Dates formatting:
df_all_idx_exported['ECO_RELEASE_DT'] = pd.to_datetime(df_all_idx_exported['ECO_RELEASE_DT'].astype(str), format = '%Y%m%d')
df_all_idx_exported['FIRST_REVISION_DATE'] = pd.to_datetime(df_all_idx_exported['FIRST_REVISION_DATE'].astype(str), format = '%Y%m%d')
df_all_idx_exported['ECO_FUTURE_RELEASE_DATE'] = pd.to_datetime(df_all_idx_exported['ECO_FUTURE_RELEASE_DATE'].astype(str), format = '%Y%m%d')
### Exported dataframe saving:
df_all_idx_exported.to_hdf(str_path_bb_idx_hdf, key = str_key_exported, mode = 'w')

In [ ]:
### RUN TO RE-EXPORT DATA: FLAGS FILE EXPORT

### Reading excel file:
df_flags = pd.read_excel(io = str_path_bb_idx_flags, sheet_name = str_flag_sheet, usecols = list(range(38))[1: ], index_col = 0, skiprows = list(range(6)), header = 0,
                               parse_dates = True, na_values = list_na_excel_values, keep_default_na = False)
### Dataframe transforming:
df_flags.dropna(how = 'all', inplace = True)
df_flags.index.name = 'Index_Name'
### Choosing columns to go on with:
df_flags = \
        df_flags[['CTRY', 'Type', 'Category', 'Negative', 'Security Description', 'SA/NSA', 'Processing', 'Index: Base', 'Type 2', 'INDX_SOURCE', 'DES_NOTES', 'INDX_FREQ']]
### Columns renaming:
df_flags.columns = ['Region', 'Type_Prime', 'Category', 'Negative', 'Description', 'SA_Status', 'Processing', 'Base', 'Type_Second', 'Data_Source', 'Notes', 'Frequency']
### Seasonality Adjustment Flag filling:
df_flags['SA_Status'] = df_flags['SA_Status'].fillna('Unclear')
### Processing Index separating:
df_flags['Processing'] = df_flags['Processing'].str.partition(': ')[0]
### Base filling:
df_flags['Base'] = df_flags['Base'].fillna(-1)
### Euro countries region renaming:
df_flags.loc[~df_flags['Region'].isin(['Japan', 'UK', 'US']), 'Region'] = 'Europe'
### Columns reordering:
df_flags = df_flags[['Type_Prime', 'Type_Second', 'Category', 'Region', 'Processing', 'Base', 'SA_Status', 'Frequency', 'Negative', 'Description', 'Data_Source', 'Notes']]
### Types converting:
df_flags = df_flags.astype({'Type_Prime': 'str',
                            'Type_Second': 'str',
                            'Category': 'str',
                            'Region': 'str',
                            'Processing': 'str',
                            'Base': 'int',
                            'SA_Status': 'str',
                            'Frequency': 'str',
                            'Negative': 'int',
                            'Description': 'str',
                            'Data_Source': 'str',
                            'Notes': 'str'})
### Exported dataframe saving:
df_flags.to_hdf(str_path_bb_idx_hdf, key = str_key_flags, mode = 'a')

In [ ]:
### RUN TO RE-EXPORT DATA: RELEASES AND REVISIONS DATA EXTRACTING

### Revisions data reading (99,180 rows):
df_all_idx_revisions = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_exported)[
                                   ['PX_LAST', 'ECO_RELEASE_DT', 'ACTUAL_RELEASE', 'FIRST_REVISION_DATE', 'FIRST_REVISION', 'ECO_FUTURE_RELEASE_DATE']]
### Dropping rows with all Index Values missed (96,369 rows left):
#df_all_idx_revisions = \
#        df_all_idx_revisions.loc[df_all_idx_revisions['PX_LAST'].notna() | df_all_idx_revisions['ACTUAL_RELEASE'].notna() | df_all_idx_revisions['FIRST_REVISION'].notna()]
### Date filtering:
df_all_idx_revisions = df_all_idx_revisions.reset_index('Data_Date')
df_all_idx_revisions = df_all_idx_revisions.loc[df_all_idx_revisions['Data_Date'] >= datetime_start].set_index('Data_Date', append = True)
### Columns renaming:
df_all_idx_revisions.columns = ['Final_Value', 'Release_Date', 'Release_Value', 'Revision_Date', 'Revision_Value', 'Future_Date']

In [ ]:
### RUN TO RE-EXPORT DATA: DATA REPAIRING

### 1 : MPMIITMA Index Revision_Value mistake correction:
df_all_idx_revisions.loc[('MPMIITMA Index', All), 'Revision_Value'] = np.NaN
### 2 : JNCPT Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('JNCPT Index', '1999-07-31'), 'Release_Date'] = pd.to_datetime('1999-07-30')
df_all_idx_revisions.loc[('JNCPT Index', '2001-01-31'), 'Release_Date'] = pd.to_datetime('2001-01-26')
df_all_idx_revisions.loc[('JNCPT Index', '2001-02-28'), 'Release_Date'] = pd.to_datetime('2001-03-02')
df_all_idx_revisions.loc[('JNCPT Index', '2001-06-30'), 'Release_Date'] = pd.to_datetime('2001-06-29')
### 3 : JNCSTOTY Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('JNCSTOTY Index', '2018-12-31'), 'Release_Date'] = pd.to_datetime('2019-01-31')
### 4: JNPIY Index Revision_Date mistakes correction:
df_all_idx_revisions.loc[('JNPIY Index', '2015-12-31'), 'Revision_Date'] = pd.to_datetime('2016-02-23')
### 5: JNPIY Index Revision_Date mistakes correction:
df_all_idx_revisions.loc[('JNTIAIAM Index', '2017-11-30'), 'Revision_Date'] = pd.to_datetime('2018-02-20')
### 6: BECPCHNG Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('BECPCHNG Index', '1997-01-31'), 'Release_Date'] = pd.to_datetime('1997-01-30')
df_all_idx_revisions.loc[('BECPCHNG Index', '2001-10-31'), 'Release_Date'] = pd.to_datetime('2001-10-28')
df_all_idx_revisions.loc[('BECPCHNG Index', '2001-11-30'), 'Release_Date'] = pd.to_datetime('2001-11-29')
df_all_idx_revisions.loc[('BECPCHNG Index', '2001-12-31'), 'Release_Date'] = pd.to_datetime('2001-12-23')
### 7 : BEUER Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('BEUER Index', '1997-01-31'), 'Release_Date'] = pd.to_datetime('1997-02-06')
### 8 : FRCPEECM Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('FRCPEECM Index', '1997-07-31'), 'Release_Date'] = pd.to_datetime('1997-08-14')
### 9 : FRCPIMOM Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('FRCPIMOM Index', '1999-01-31'), 'Release_Date'] = pd.to_datetime('1999-02-24')
df_all_idx_revisions.loc[('FRCPIMOM Index', '1999-02-28'), 'Release_Date'] = pd.to_datetime('1999-03-24')
df_all_idx_revisions.loc[('FRCPIMOM Index', '2006-01-31'), 'Release_Date'] = pd.to_datetime('2006-02-21')
### 10 : GKCPNEWY Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-03-31'), 'Release_Date'] = pd.to_datetime('2001-04-09')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-04-30'), 'Release_Date'] = pd.to_datetime('2001-05-10')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-05-31'), 'Release_Date'] = pd.to_datetime('2001-06-10')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-06-30'), 'Release_Date'] = pd.to_datetime('2001-07-05')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-07-31'), 'Release_Date'] = pd.to_datetime('2001-08-09')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-08-31'), 'Release_Date'] = pd.to_datetime('2001-09-09')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-09-30'), 'Release_Date'] = pd.to_datetime('2001-10-08')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-10-31'), 'Release_Date'] = pd.to_datetime('2001-11-08')
df_all_idx_revisions.loc[('GKCPNEWY Index', '2001-11-30'), 'Release_Date'] = pd.to_datetime('2001-12-09')
### 11 : GKUERATE Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('GKUERATE Index', '2007-01-31'), 'Release_Date'] = np.NaN
### 12 : GRCP2NRM Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('GRCP2NRM Index', '2006-01-31'), 'Release_Date'] = pd.to_datetime('2006-01-30')
### 13 : GRCP2SAM Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('GRCP2SAM Index', '2006-01-31'), 'Release_Date'] = pd.to_datetime('2006-01-30')
### 14 : ITVHYOY Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('ITVHYOY Index', '1999-01-31'), 'Release_Date'] = pd.to_datetime('1999-02-03')
### 15 : JNLSUCTL Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('JNLSUCTL Index', '2003-01-31'), 'Release_Date'] = pd.to_datetime('2003-03-01')
### 16 : PTCIEC Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('PTCIEC Index', '2007-12-31'), 'Release_Date'] = pd.to_datetime('2008-01-08')
### 17 : SLCPLHMM Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('SLCPLHMM Index', '2004-04-30'), 'Release_Date'] = pd.to_datetime('2004-05-20')
df_all_idx_revisions.loc[('SLCPLHMM Index', '2005-01-31'), 'Release_Date'] = pd.to_datetime('2005-02-27')
df_all_idx_revisions.loc[('SLCPLHMM Index', '2005-02-28'), 'Release_Date'] = pd.to_datetime('2005-03-16')
### 18 : SPCPEUMM Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-01-31'), 'Release_Date'] = pd.to_datetime('2000-02-13')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-02-29'), 'Release_Date'] = pd.to_datetime('2000-03-14')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-03-31'), 'Release_Date'] = pd.to_datetime('2000-04-11')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-04-30'), 'Release_Date'] = pd.to_datetime('2000-05-11')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-05-31'), 'Release_Date'] = pd.to_datetime('2000-06-13')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-06-30'), 'Release_Date'] = pd.to_datetime('2000-07-13')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-07-31'), 'Release_Date'] = pd.to_datetime('2000-08-14')
df_all_idx_revisions.loc[('SPCPEUMM Index', '2000-08-31'), 'Release_Date'] = pd.to_datetime('2000-09-13')
### 19 : SPROCHNG Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('SPROCHNG Index', '2008-01-31'), 'Release_Date'] = pd.to_datetime('2008-02-25')
### 20 : SVAWRYOY Index Release_Date mistakes correction:
df_all_idx_revisions.loc[('SVAWRYOY Index', '2006-08-31'), 'Release_Date'] = pd.to_datetime('2006-10-15')
df_all_idx_revisions.loc[('SVAWRYOY Index', '2006-09-30'), 'Release_Date'] = pd.to_datetime('2006-11-15')
df_all_idx_revisions.loc[('SVAWRYOY Index', '2006-10-31'), 'Release_Date'] = pd.to_datetime('2006-12-17')
### 21 : INJCJC Index Data Date shifts correction:
df_all_idx_revisions.loc[('INJCJC Index', '2020-08-28'), 'Release_Date'] = np.NaN
str_ei_to_correct = 'INJCJC Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_to_correct = df_ei_to_correct.drop(df_ei_to_correct.loc[df_ei_to_correct.isna().all(axis = 1)].index)
df_ei_to_correct_shifted = df_ei_to_correct.shift(-1)
df_ei_to_correct_shifted.columns = [str_column_name + '_shifted' for str_column_name in df_ei_to_correct_shifted.columns] 
df_ei_corrected = pd.concat([df_ei_to_correct, df_ei_to_correct_shifted], axis = 1)
df_ei_corrected.loc[df_ei_corrected['Final_Value_shifted'].isna() & df_ei_corrected['Revision_Value_shifted'].notna(), 'Revision_Date'] = \
                df_ei_corrected['Revision_Date_shifted']
df_ei_corrected.loc[df_ei_corrected['Final_Value_shifted'].isna() & df_ei_corrected['Revision_Value_shifted'].notna(), 'Revision_Value'] = \
                df_ei_corrected['Revision_Value_shifted']
df_ei_corrected = df_ei_corrected[df_ei_to_correct.columns]
df_ei_corrected = df_ei_corrected.drop(df_ei_corrected[df_ei_corrected['Final_Value'].isna()].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 22 : INJCSP Index Data Date shifts correction:
str_ei_to_correct = 'INJCSP Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct['Final_Value'].isna()].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 23 : ITSR1B Index Data Date shifts correction: 
df_all_idx_revisions = df_all_idx_revisions.drop(index = ('ITSR1B Index', '1992-12-31'))
### 24 : ATIPIMM Index partly dropping due to the gap:
str_ei_to_correct = 'ATIPIMM Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('2004-04-30')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 25 : ATIPIMM Index partly dropping due to the gap:
str_ei_to_correct = 'GKIPIYOY Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('2001-01-31')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 26 : IEIPIMOM Index partly dropping due to the gap:
str_ei_to_correct = 'IEIPIMOM Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('2001-02-28')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 27 : ITNHMOM Index partly dropping due to the gap:
str_ei_to_correct = 'ITNHMOM Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('2003-03-31')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 28 : MPMIGBXA Index partly dropping due to the gap:
str_ei_to_correct = 'MPMIGBXA Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('2005-07-31')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 29 : PCE CHNC Index partly dropping due to the gap:
str_ei_to_correct = 'PCE CHNC Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('1999-02-28')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 30 : PTPPMOM Index partly dropping due to the gap:
str_ei_to_correct = 'PTPPMOM Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('1999-02-28')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 31 : SNTEEUGX Index partly dropping due to the gap:
str_ei_to_correct = 'SNTEEUGX Index'
df_ei_to_correct = df_all_idx_revisions.loc[([str_ei_to_correct], All), All]
df_ei_corrected = df_ei_to_correct.drop(df_ei_to_correct[df_ei_to_correct.index.get_level_values('Data_Date') < pd.to_datetime('2003-01-31')].index)
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_corrected], axis = 0).sort_index()
### 32 : UKISCTMM Index Data Date shifts correction: 
df_all_idx_revisions = df_all_idx_revisions.drop(index = ('UKISCTMM Index', '1992-12-31'))
### 33 : IERSVMOM Index adding missed rows:
str_ei_to_correct = 'IERSVMOM Index'
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2001-01-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2001-02-28')), All] = np.NaN
### 34 : ITNHMOM Index adding missed rows:
str_ei_to_correct = 'ITNHMOM Index'
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2020-07-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2020-08-31')), All] = np.NaN
### 35 : NEISIYOY Index adding missed rows:
str_ei_to_correct = 'NEISIYOY Index'
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('1999-07-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('1999-08-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2000-07-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2000-11-30')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2001-07-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2002-07-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2003-07-31')), All] = np.NaN
### 36 : NEISIYOY Index adding missed rows:
str_ei_to_correct = 'MAPMINDX Index'
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2016-05-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2016-06-30')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2016-07-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2016-08-31')), All] = np.NaN
df_all_idx_revisions.loc[(str_ei_to_correct, pd.to_datetime('2016-09-30')), All] = np.NaN
### 37 : COMFCOMF Index values correcting:
str_ei_to_correct = 'COMFCOMF Index'
df_ei_to_correct = df_all_idx_revisions.copy().loc[(str_ei_to_correct, All), All]
df_ei_to_correct.loc[df_ei_to_correct['Release_Value'] < 10, 'Release_Value'] = (1 + df_ei_to_correct['Release_Value'] / 100) * 50
df_ei_to_correct.loc[df_ei_to_correct['Revision_Value'] < 10, 'Revision_Value'] = (1 + df_ei_to_correct['Revision_Value'] / 100) * 50
df_all_idx_revisions = df_all_idx_revisions.drop(str_ei_to_correct, level = 'Index_Name')
df_all_idx_revisions = pd.concat([df_all_idx_revisions, df_ei_to_correct], axis = 0).sort_index()

In [ ]:
### RUN TO TESTING: RELEASE DATE < DATA DATE

df_less_data_date = df_all_idx_revisions.loc[df_all_idx_revisions['Release_Date'] < df_all_idx_revisions.index.get_level_values(1) - pd.offsets.BDay(15)]
print(df_less_data_date.index.get_level_values('Index_Name').unique())

In [ ]:
### RUN TO TESTING: REVISION DATE < RELEASE DATE

df_less_release_date = df_all_idx_revisions.loc[df_all_idx_revisions['Revision_Date'] < df_all_idx_revisions['Release_Date']]
print(df_less_release_date.index.get_level_values('Index_Name').unique())
print(df_less_release_date.loc[(All, All), All])

In [ ]:
### RUN TO TESTING: RELEASE DATE >> DATA DATE

df_less_data_date = df_all_idx_revisions.loc[df_all_idx_revisions['Release_Date'] > df_all_idx_revisions.index.get_level_values(1) + pd.offsets.BDay(150)]
print(df_less_data_date.index.get_level_values('Index_Name').unique())
print(df_less_data_date.loc[('ASPPIMOM Index', All), All])

In [ ]:
### RUN TO TESTING: RELEASE DATE > NEXT RELEASE DATE

df_released_date = df_all_idx_revisions['Release_Date'].to_frame()
df_released_date['Release_Date_next'] = df_released_date['Release_Date'].groupby('Index_Name').transform(lambda ser_eco_ind: ser_eco_ind.shift(-1))
df_less_release_date = df_released_date.loc[df_released_date['Release_Date_next'] < df_released_date['Release_Date']]
print(df_less_release_date.index.get_level_values('Index_Name').unique())
print(df_less_release_date.loc[(All, All), All])

In [ ]:
### RUN TO TESTING: EXTRA SHORT INTERVALS

### Frequency check:
ser_data_date = df_all_idx_revisions.reset_index('Data_Date')['Data_Date'].squeeze()
df_eco_ind_agg = ser_data_date.groupby('Index_Name').apply(lambda ser_eco_ind: (ser_eco_ind - ser_eco_ind.shift()).dt.days)\
                               .groupby('Index_Name').agg(['min', 'max', 'median'])
print('Frequency control: Extra minimum:\n', df_eco_ind_agg.loc[df_eco_ind_agg['median'] / 1.2 > df_eco_ind_agg['min']])

In [ ]:
### RUN TO TESTING: EXTRA LONG INTERVALS

### Frequency check:
ser_data_date = df_all_idx_revisions.reset_index('Data_Date')['Data_Date'].squeeze()
df_eco_ind_agg = ser_data_date.groupby('Index_Name').apply(lambda ser_eco_ind: (ser_eco_ind - ser_eco_ind.shift()).dt.days)\
                               .groupby('Index_Name').agg(['min', 'max', 'median'])
print('Frequency control: Extra maximum:\n', df_eco_ind_agg.loc[df_eco_ind_agg['median'] * 3 < df_eco_ind_agg['max']])

In [ ]:
### RUN TO TESTING: DATA QUALITY AND COMPLETENESS

### Release Date <= Event Date:
print('Overall observations number:', len(df_all_idx_revisions.index))
df_less_data_date = df_all_idx_revisions.loc[df_all_idx_revisions['Release_Date'] < df_all_idx_revisions.index.get_level_values(1)]
print('Observations with Release Date < Data Date number:', len(df_less_data_date.index))
print('Unique Eco Indices with Release Date < Data Date cases number:', len(df_less_data_date.index.get_level_values(0).unique()))
### Release Date vs First Revision Date:
print('Release Date > First Revision Date:\n', 
      df_all_idx_revisions.loc[df_all_idx_revisions['Release_Date'] > df_all_idx_revisions['Revision_Date'], ['Release_Date', 'Revision_Date']])
### Release Date vs next Release Date:
df_released_date = df_all_idx_revisions['Release_Date'].to_frame()
df_released_date['Release_Date_shifted'] = df_released_date['Release_Date'].groupby('Index_Name').transform(lambda ser_eco_ind: ser_eco_ind.shift(-1))
print('Release Date > next Release Date:\n', df_released_date.loc[df_released_date['Release_Date'] > df_released_date['Release_Date_shifted']])
### Frequency check:
ser_data_date = df_all_idx_revisions.reset_index('Data_Date')['Data_Date'].squeeze()
df_eco_ind_agg = ser_data_date.groupby('Index_Name').apply(lambda ser_eco_ind: (ser_eco_ind - ser_eco_ind.shift()).dt.days)\
                               .groupby('Index_Name').agg(['min', 'max', 'median'])
print('Frequency control: Extra short intervals:\n', df_eco_ind_agg.loc[df_eco_ind_agg['median'] / 1.2 > df_eco_ind_agg['min']])
print('Frequency control: Extra long intervals:\n', df_eco_ind_agg.loc[df_eco_ind_agg['median'] * 2 < df_eco_ind_agg['max']])

In [ ]:
### RUN TO RE-EXPORT DATA: EXTRACTING Release and first revision date/value pairs:

### Release data extracting:
df_release_based = df_all_idx_revisions[['Release_Date', 'Release_Value']]
df_release_based.columns = ['Observation_Date', 'Index_Value']
### All empty release date vector Eco Indices:
ser_empty_release_date = df_release_based['Observation_Date'].groupby('Index_Name').count()
ser_empty_release_date = ser_empty_release_date[ser_empty_release_date == 0]
### All empty release value vector Eco Indices:
ser_empty_release_value = df_release_based['Index_Value'].groupby('Index_Name').count()
ser_empty_release_value = ser_empty_release_value[ser_empty_release_value == 0]
### All empty release date or all empty release value indices list:
list_empty_release = sorted(list(set(ser_empty_release_date.index).union(set(ser_empty_release_value.index))))
### Revision data extracting:
df_revision_based = df_all_idx_revisions[['Revision_Date', 'Revision_Value']]
df_revision_based.columns = ['Observation_Date', 'Index_Value']
### All empty first revision date vector Eco Indices:
ser_empty_revision_date = df_revision_based['Observation_Date'].groupby('Index_Name').count()
ser_empty_revision_date = ser_empty_revision_date[ser_empty_revision_date == 0]
### All empty first revision value vector Eco Indices:
ser_empty_revision_value = df_revision_based['Index_Value'].groupby('Index_Name').count()
ser_empty_revision_value = ser_empty_revision_value[ser_empty_revision_value == 0]
### All empty first revision date or all empty first revision value indices list:
list_empty_revision = sorted(list(set(ser_empty_revision_date.index).union(set(ser_empty_revision_value.index))))

In [ ]:
### RUN TO TESTING: MAIN STATS

### General stats:
print('Rows with at least one value filled:', len(df_all_idx_revisions.index))
print('Eco Indices number:', len(df_all_idx_revisions.index.get_level_values(0).unique()))
print('Average data dates number:', int(df_all_idx_revisions.groupby('Index_Name').apply(lambda df_group: len(df_group.index)).mean()))
### Release data analyzing (96,369 rows):
### Data quantity:
print('Release Info: filled date & value:', len(df_release_based.loc[df_release_based['Observation_Date'].notna() & df_release_based['Index_Value'].notna()]), 'rows')
print('Release Info: empty date & value:', len(df_release_based.loc[df_release_based['Observation_Date'].isna() & df_release_based['Index_Value'].isna()]), 'rows')
print('Release Info: empty date & filled value:', len(df_release_based.loc[df_release_based['Observation_Date'].isna() & df_release_based['Index_Value'].notna()]), 'rows')
print('Release Info: filled date & empty value:', len(df_release_based.loc[df_release_based['Observation_Date'].notna() & df_release_based['Index_Value'].isna()]), 'rows')
### First revision data analyzing (96,369 rows):
### Data quantity:
print('First revision Info: filled date & value:', 
      len(df_revision_based.loc[df_revision_based['Observation_Date'].notna() & df_revision_based['Index_Value'].notna()]), 'rows')
print('First revision Info: empty date & value:', 
      len(df_revision_based.loc[df_revision_based['Observation_Date'].isna() & df_revision_based['Index_Value'].isna()]), 'rows')
print('First revision Info: empty date & filled value:', 
      len(df_revision_based.loc[df_revision_based['Observation_Date'].isna() & df_revision_based['Index_Value'].notna()]), 'rows')
print('First revision Info: filled date & empty value:', 
      len(df_revision_based.loc[df_revision_based['Observation_Date'].notna() & df_revision_based['Index_Value'].isna()]), 'rows') 

In [ ]:
### RUN TO TESTING: DETAILED STATS: RELEASE DATE/VALUE PAIR

print('Eco Indices without any release date number:', len(ser_empty_release_date.index))
print('Eco Indices without any release value number:', len(ser_empty_release_value.index))
### Empty Eco Indices relese info lists comparision:
print('Symmetric difference of empty release info Eco Indices lists:', list(set(ser_empty_release_date.index).symmetric_difference(set(ser_empty_release_value.index))))
print('Union of empty release info Eco Indices lists:\n', sorted(list(set(ser_empty_release_date.index).union(set(ser_empty_release_value.index)))))
for iter_eco_index in sorted(list(set(ser_empty_release_date.index).symmetric_difference(set(ser_empty_release_value.index)))):
    print('Symmetric difference for', iter_eco_index, ':\n', 
          df_all_idx_revisions.loc[(iter_eco_index, All), All].dropna(subset = ['Release_Date', 'Release_Value'], how = 'all'))

In [ ]:
### RUN TO TESTING: DETAILED STATS: FIRST REVISION DATE/VALUE PAIR

print('Eco Indices without any revision date number:', len(ser_empty_revision_date.index))
print('Eco Indices without any revision value number:', len(ser_empty_revision_value.index))
### Empty Eco Indices relese info lists comparision:
print('Symmetric difference of empty revision info Eco Indices lists:', 
      sorted(list(set(ser_empty_revision_date.index).symmetric_difference(set(ser_empty_revision_value.index)))))
print('Union of empty revision info Eco Indices lists:\n', sorted(list(set(ser_empty_revision_date.index).union(set(ser_empty_revision_value.index)))))
for iter_eco_index in sorted(list(set(ser_empty_revision_date.index).symmetric_difference(set(ser_empty_revision_value.index)))):
    print('Symmetric difference for', iter_eco_index, ':\n', 
          df_all_idx_revisions.loc[(iter_eco_index, All), All].dropna(subset = ['Revision_Date', 'Revision_Value'], how = 'all'))

In [ ]:
### RUN TO RE-EXPORT DATA: ALL DATES FILLING PROCEDURE

df_flags = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_flags)

#### Dates filling:
def all_dates_filler(df_eco_ind):
    ### Eco Indice options saving:
    str_index_name = df_eco_ind.index.get_level_values(0)[0]
    str_eco_ind_freq = df_flags.loc[str_index_name, 'Frequency']
    if (str_eco_ind_freq in dict_final_only_lag.keys()):
        int_final_only_lag = dict_final_only_lag[str_eco_ind_freq]
    else:
        int_final_only_lag = dict_final_only_lag['Other']
    ### Index_Name level dropping:
    df_eco_ind = df_eco_ind.droplevel(0)
    ### Index duplicating to column:
    df_eco_ind['Event_Date'] = df_eco_ind.index
    df_eco_ind['Release_Lag'] = np.NaN
    ### No release checking:
    if (str_index_name in list_empty_release): 
        df_eco_ind['Final_Date'] = df_eco_ind['Event_Date'] + pd.Timedelta(int_final_only_lag, 'D')
    elif (str_index_name in list_empty_revision):    
        ### Backfilling release dates lag before the first known released date: 
        idx_first_valid_release = df_eco_ind['Release_Date'].first_valid_index()
        df_first_released = df_eco_ind.dropna(subset = ['Release_Date']).iloc[ : int_first_mean_length]        
        int_first_release_mean = (df_first_released['Release_Date'] - df_first_released['Event_Date']).mean().days + 1
        df_eco_ind.loc[ : idx_first_valid_release, 'Release_Lag'] = int_first_release_mean
        ### Interpolating missed release dates lag after the first known released date:
        df_eco_ind.loc[idx_first_valid_release : , 'Release_Lag'] = (df_eco_ind['Release_Date'] - df_eco_ind['Event_Date']).dt.days
        df_eco_ind.loc[idx_first_valid_release : , 'Release_Lag'] = df_eco_ind.loc[idx_first_valid_release : , 'Release_Lag'].interpolate(method = 'linear').round()
        ### Filling empty release date by lag adding:
        df_eco_ind['Release_Lag'] = pd.to_timedelta(df_eco_ind['Release_Lag'], 'D')    
        df_eco_ind.loc[df_eco_ind['Release_Date'].isna(), 'Release_Date'] = df_eco_ind['Event_Date'] + df_eco_ind['Release_Lag']
        ### Final date filling as one event shifted release date
        df_eco_ind['Final_Date'] = df_eco_ind['Release_Date'].shift(-int_revision_shift)   
    else:
        ### Backfilling release dates lag before the first known released date: 
        idx_first_valid_release = df_eco_ind['Release_Date'].first_valid_index()
        df_first_released = df_eco_ind.dropna(subset = ['Release_Date']).iloc[ : int_first_mean_length]     
        int_first_release_mean = (df_first_released['Release_Date'] - df_first_released['Event_Date']).mean().days + 1        
        df_eco_ind.loc[ : idx_first_valid_release, 'Release_Lag'] = int_first_release_mean
        ### Interpolating missed release dates lag after the first known released date:
        df_eco_ind.loc[idx_first_valid_release : , 'Release_Lag'] = (df_eco_ind['Release_Date'] - df_eco_ind['Event_Date']).dt.days
        df_eco_ind.loc[idx_first_valid_release : , 'Release_Lag'] = df_eco_ind.loc[idx_first_valid_release : , 'Release_Lag'].interpolate(method = 'linear').round()
        ### Filling empty release date with lag adding:
        df_eco_ind['Release_Lag'] = pd.to_timedelta(df_eco_ind['Release_Lag'], 'D')    
        df_eco_ind.loc[df_eco_ind['Release_Date'].isna(), 'Release_Date'] = df_eco_ind['Event_Date'] + df_eco_ind['Release_Lag']
        ### Filling empty revision dates: 
        df_eco_ind.loc[df_eco_ind['Revision_Date'].isna(), 'Revision_Date'] = df_eco_ind['Release_Date'].shift(-int_revision_shift)
        ### Final date filling as one event shifted release date
        df_eco_ind['Final_Date'] = df_eco_ind['Release_Date'].shift(-int_final_shift)
    ### Last values having revision date with no revision value managing:
    idx_last_values = df_eco_ind['Revision_Date'].notna() & df_eco_ind['Revision_Value'].isna() & df_eco_ind['Final_Date'].isna() & df_eco_ind['Final_Value'].notna() 
    df_eco_ind.loc[idx_last_values, 'Revision_Value'] = df_eco_ind['Final_Value']
    df_eco_ind.loc[idx_last_values, 'Final_Value'] = np.NaN
    ### Last values having release date with no release value managing:
    idx_last_values = df_eco_ind['Release_Date'].notna() & df_eco_ind['Release_Value'].isna() & df_eco_ind['Final_Date'].isna() & df_eco_ind['Final_Value'].notna() 
    df_eco_ind.loc[idx_last_values, 'Release_Value'] = df_eco_ind['Final_Value']
    df_eco_ind.loc[idx_last_values, 'Final_Value'] = np.NaN
    ### Last values having final value with no final date and future date managing (no news detected):
    idx_last_values = df_eco_ind['Future_Date'].isna() & df_eco_ind['Final_Date'].isna() & df_eco_ind['Final_Value'].notna() 
    df_eco_ind.loc[idx_last_values, 'Final_Value'] = np.NaN
    ### Last values having final value with no final date and having revision pair managing (no news detected):
    idx_last_values = df_eco_ind['Revision_Date'].notna() & df_eco_ind['Revision_Value'].notna() & df_eco_ind['Final_Date'].isna() & df_eco_ind['Final_Value'].notna() 
    df_eco_ind.loc[idx_last_values, 'Final_Value'] = np.NaN
    ### Last values having final value with no final date and no revision pair managing:
    idx_last_values = df_eco_ind['Revision_Date'].isna() & df_eco_ind['Revision_Value'].isna() & df_eco_ind['Final_Date'].isna() & df_eco_ind['Final_Value'].notna() 
    df_eco_ind.loc[idx_last_values, 'Final_Date'] = df_eco_ind['Future_Date']    
    ### Results ouput:
    return df_eco_ind[['Release_Date', 'Release_Value', 'Revision_Date', 'Revision_Value', 'Final_Date', 'Final_Value']]

### Date filling for each eco index performing:
df_dates_filled = df_all_idx_revisions.groupby('Index_Name').apply(all_dates_filler)

In [ ]:
### RUN TO RE-EXPORT DATA: DATA REPAIRING AND SAVING

### 38 : GKCPIUHY Index Final Date shifts correction: 
df_dates_filled.loc[('GKCPIUHY Index', '2003-10-31'), 'Final_Date'] = pd.to_datetime('2004-02-17')
df_dates_filled.loc[('GKCPIUHY Index', '2003-11-30'), 'Final_Date'] = pd.to_datetime('2004-02-17')
### 39 : GKCPNEWY Index Final Date shifts correction: 
df_dates_filled.loc[('GKCPNEWY Index', '2003-10-31'), 'Final_Date'] = pd.to_datetime('2004-02-10')
df_dates_filled.loc[('GKCPNEWY Index', '2003-11-30'), 'Final_Date'] = pd.to_datetime('2004-02-10')
### 40 : EUCPTSAM Index Final Date shifts correction: 
df_dates_filled.loc[('EUCPTSAM Index', '2004-12-31'), 'Final_Date'] = pd.to_datetime('2005-09-12')
df_dates_filled.loc[('EUCPTSAM Index', '2005-03-31'), 'Final_Date'] = pd.to_datetime('2005-12-09')
df_dates_filled.loc[('EUCPTSAM Index', '2005-06-30'), 'Final_Date'] = pd.to_datetime('2006-03-10')
df_dates_filled.loc[('EUCPTSAM Index', '2005-09-30'), 'Final_Date'] = pd.to_datetime('2006-06-09')
df_dates_filled.loc[('EUCPTSAM Index', '2005-12-31'), 'Final_Date'] = pd.to_datetime('2006-09-06')
df_dates_filled.loc[('EUCPTSAM Index', '2006-03-31'), 'Final_Date'] = pd.to_datetime('2006-12-06')
df_dates_filled.loc[('EUCPTSAM Index', '2006-06-30'), 'Final_Date'] = pd.to_datetime('2006-12-06')
df_dates_filled.loc[('EUCPTSAM Index', '2006-09-30'), 'Final_Date'] = pd.to_datetime('2007-01-18')
### 41 : SVUER Index Revision Date shifts correction: 
df_dates_filled.loc[('SVUER Index', '2005-06-30'), 'Revision_Date'] = pd.to_datetime('2005-09-18')
### 42 : UKMPIMOM Index Revision Date shifts correction: 
df_dates_filled.loc[('UKMPIMOM Index', '2001-01-31'), 'Revision_Date'] = pd.to_datetime('2001-04-09')
df_dates_filled.loc[('UKMPIMOM Index', '2001-02-28'), 'Revision_Date'] = pd.to_datetime('2001-05-08')
df_dates_filled.loc[('UKMPIMOM Index', '2001-03-31'), 'Revision_Date'] = pd.to_datetime('2001-06-11')
df_dates_filled.loc[('UKMPIMOM Index', '2001-04-30'), 'Revision_Date'] = pd.to_datetime('2001-07-10')
df_dates_filled.loc[('UKMPIMOM Index', '2001-05-31'), 'Revision_Date'] = pd.to_datetime('2001-08-05')
df_dates_filled.loc[('UKMPIMOM Index', '2001-08-31'), 'Revision_Date'] = pd.to_datetime('2001-11-05')
df_dates_filled.loc[('UKMPIMOM Index', '2001-10-31'), 'Revision_Date'] = pd.to_datetime('2002-01-14')
### 43 : ESCPLMOM Index early values clearing: 
df_dates_filled.loc[('ESCPLMOM Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(1994, 1, 1)), All] = np.NaN
### 44 : ESCPLMOM Index early values clearing: 
df_dates_filled.loc[('ESPPIMOM Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(2003, 1, 1)), All] = np.NaN
### 45 : ESCPLMOM Index revisions clearing:
df_dates_filled.loc[('ESPPIMOM Index', All), 'Revision_Value'] = np.NaN
### 46 : IECPIMOM Index early values clearing:
df_dates_filled.loc[('IECPIMOM Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(1998, 1, 1)), All] = np.NaN
### 47 : JBTARATE Index early values clearing:
df_dates_filled.loc[('JBTARATE Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(1990, 1, 1)), All] = np.NaN
### 48 : SVBTSI Index early values clearing:
df_dates_filled.loc[('SVBTSI Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(2005, 1, 1)), All] = np.NaN
### 49 : SVIPTMOM Index early values clearing:
df_dates_filled.loc[('SVIPTMOM Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(2008, 1, 1)), All] = np.NaN
### 50 : UKPPIINC Index early values clearing:
df_dates_filled.loc[('UKPPIINC Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(1997, 1, 1)), All] = np.NaN
### 51 : UKPPXFBM Index early values clearing:
df_dates_filled.loc[('UKPPXFBM Index', df_dates_filled.index.get_level_values('Data_Date') < datetime(1997, 1, 1)), All] = np.NaN
### 52 : GRCP2BRM Index outlier values clipping:
df_dates_filled.loc[('GRCP2BRM Index', df_dates_filled['Final_Value'] < -2), 'Final_Value'] = -2.0
### 53 : GRCP2SAM Index outlier values clipping:
df_dates_filled.loc[('GRCP2SAM Index', df_dates_filled['Final_Value'] < -5), 'Final_Value'] = -5.0
df_dates_filled.loc[('GRCP2SAM Index', df_dates_filled['Final_Value'] > 5), 'Final_Value'] = 5.0
### 54 : PITLCHNG Index outlier values clipping:
df_dates_filled.loc[('PITLCHNG Index', df_dates_filled['Release_Value'] < -2), 'Release_Value'] = -2.0
df_dates_filled.loc[('PITLCHNG Index', df_dates_filled['Release_Value'] > 2), 'Release_Value'] = 2.0
df_dates_filled.loc[('PITLCHNG Index', df_dates_filled['Revision_Value'] < -2), 'Revision_Value'] = -2.0
df_dates_filled.loc[('PITLCHNG Index', df_dates_filled['Revision_Value'] > 2), 'Revision_Value'] = 2.0
df_dates_filled.loc[('PITLCHNG Index', df_dates_filled['Final_Value'] < -2), 'Final_Value'] = -2.0
df_dates_filled.loc[('PITLCHNG Index', df_dates_filled['Final_Value'] > 2), 'Final_Value'] = 2.0
### 55 : ITNSSTN Index outlier values clipping:
df_dates_filled.loc[('ITNSSTN Index', df_dates_filled['Release_Value'] < -10), 'Release_Value'] = -10.0
df_dates_filled.loc[('ITNSSTN Index', df_dates_filled['Release_Value'] > 10), 'Release_Value'] = 10.0
df_dates_filled.loc[('ITNSSTN Index', df_dates_filled['Revision_Value'] < -10), 'Revision_Value'] = -10.0
df_dates_filled.loc[('ITNSSTN Index', df_dates_filled['Revision_Value'] > 10), 'Revision_Value'] = 10.0
df_dates_filled.loc[('ITNSSTN Index', df_dates_filled['Final_Value'] < -10), 'Final_Value'] = -10.0
df_dates_filled.loc[('ITNSSTN Index', df_dates_filled['Final_Value'] > 10), 'Final_Value'] = 10.0
### 56 : EUNOEUM Index dropping:
df_dates_filled.drop(labels = 'ESRSREHY Index', level = 'Index_Name', inplace = True)
### 57 : EUNOEUM Index dropping:
df_dates_filled.drop(labels = 'EUNOEUM Index', level = 'Index_Name', inplace = True)
### 58 : NEUETOTR Index dropping:
df_dates_filled.drop(labels = 'NEUETOTR Index', level = 'Index_Name', inplace = True)
### 59 : RSSA25M Index dropping:
df_dates_filled.drop(labels = 'RSSA25M Index', level = 'Index_Name', inplace = True)
### 60 : UKISCTMM Index dropping:
df_dates_filled.drop(labels = 'UKISCTMM Index', level = 'Index_Name', inplace = True)
### 61 : UKMSVTVJ Index dropping:
df_dates_filled.drop(labels = 'UKMSVTVJ Index', level = 'Index_Name', inplace = True)
### 62 : UKNCCCIS Index dropping:
df_dates_filled.drop(labels = 'UKNCCCIS Index', level = 'Index_Name', inplace = True)
### 63 : SVUER Index revisions clearing:
df_dates_filled.loc[('SVUER Index', All), 'Revision_Value'] = np.NaN
### 64 : CPTICHNG Index finals clearing:
df_dates_filled.loc[('CPTICHNG Index', All), 'Final_Value'] = np.NaN
### 65 : ECO1GFKC Index finals clearing:
df_dates_filled.loc[('ECO1GFKC Index', All), 'Final_Value'] = np.NaN
### 66 : JNPIY Index finals clearing:
df_dates_filled.loc[('JNPIY Index', All), 'Final_Value'] = np.NaN
### 67 : PTCCI Index finals clearing:
df_dates_filled.loc[('PTCCI Index', All), 'Final_Value'] = np.NaN
### 68 : PTPPMOM Index finals clearing:
df_dates_filled.loc[('PTPPMOM Index', All), 'Final_Value'] = np.NaN
### 69 : UKUER Index finals clearing:
df_dates_filled.loc[('UKUER Index', All), 'Final_Value'] = np.NaN
### 70 : UMRTEMU Index finals clearing:
df_dates_filled.loc[('UMRTEMU Index', All), 'Final_Value'] = np.NaN
### 71 : GRIFPBUS Index finals partly clearing and releases/revisions partly shifting:
date_break = datetime(2018, 3, 31)
flo_shift = (df_dates_filled.loc[('GRIFPBUS Index', [date_break]), 'Release_Value'] - df_dates_filled.loc[('GRIFPBUS Index', [date_break]), 'Revision_Value']).values[0]
df_dates_filled.loc[('GRIFPBUS Index', df_dates_filled.index.get_level_values('Data_Date') < date_break), 'Final_Value'] = np.NaN
df_dates_filled.loc[('GRIFPBUS Index', df_dates_filled.index.get_level_values('Data_Date') <= date_break), 'Release_Value'] = \
        df_dates_filled.loc[('GRIFPBUS Index', df_dates_filled.index.get_level_values('Data_Date') <= date_break), 'Release_Value'] - flo_shift
df_dates_filled.loc[('GRIFPBUS Index', df_dates_filled.index.get_level_values('Data_Date') < date_break), 'Revision_Value'] = \
        df_dates_filled.loc[('GRIFPBUS Index', df_dates_filled.index.get_level_values('Data_Date') < date_break), 'Revision_Value'] - flo_shift
### 72 : GRIFPCA Index finals partly clearing and releases/revisions partly shifting:
date_break = datetime(2018, 3, 31)
flo_shift = (df_dates_filled.loc[('GRIFPCA Index', [date_break]), 'Release_Value'] - df_dates_filled.loc[('GRIFPCA Index', [date_break]), 'Revision_Value']).values[0]
df_dates_filled.loc[('GRIFPCA Index', df_dates_filled.index.get_level_values('Data_Date') < date_break), 'Final_Value'] = np.NaN
df_dates_filled.loc[('GRIFPCA Index', df_dates_filled.index.get_level_values('Data_Date') <= date_break), 'Release_Value'] = \
        df_dates_filled.loc[('GRIFPCA Index', df_dates_filled.index.get_level_values('Data_Date') <= date_break), 'Release_Value'] - flo_shift
df_dates_filled.loc[('GRIFPCA Index', df_dates_filled.index.get_level_values('Data_Date') < date_break), 'Revision_Value'] = \
        df_dates_filled.loc[('GRIFPCA Index', df_dates_filled.index.get_level_values('Data_Date') < date_break), 'Revision_Value'] - flo_shift
### 73 : FRJSTCHG Index releases/revisions clearing:
df_dates_filled.loc[('FRJSTCHG Index', All), ['Release_Value', 'Revision_Value']] = np.NaN
### 74 : JNCAP Index releases/revisions clearing:
df_dates_filled.loc[('JNCAP Index', All), ['Release_Value', 'Revision_Value']] = np.NaN
### 75 : IEUERT Index releases/revisions clearing:
df_dates_filled.loc[('IEUERT Index', df_dates_filled.index.get_level_values('Data_Date') <= '2000-01-01'), ['Release_Value', 'Revision_Value']] = np.NaN
### 76 : JNCICCOI Index releases/revisions clearing:
df_dates_filled.loc[('JNCICCOI Index', df_dates_filled.index.get_level_values('Data_Date') <= '2020-01-01'), ['Release_Value', 'Revision_Value']] = np.NaN
### Empty Dates dropping:
df_dates_filled.loc[df_dates_filled['Release_Date'].notna() & df_dates_filled['Release_Value'].isna(), 'Release_Date'] = np.NaN
df_dates_filled.loc[df_dates_filled['Revision_Date'].notna() & df_dates_filled['Revision_Value'].isna(), 'Revision_Date'] = np.NaN
df_dates_filled.loc[df_dates_filled['Final_Date'].notna() & df_dates_filled['Final_Value'].isna(), 'Final_Date'] = np.NaN
### Moving early release dates to data dates:
df_dates_filled.loc[df_dates_filled.index.get_level_values('Data_Date') > df_dates_filled['Release_Date'], 'Release_Date'] = \
    df_dates_filled.loc[df_dates_filled.index.get_level_values('Data_Date') > df_dates_filled['Release_Date']].index.get_level_values('Data_Date')
### Moving early revision dates to data dates:
df_dates_filled.loc[df_dates_filled['Revision_Date'] < df_dates_filled['Release_Date'], 'Revision_Date'] = \
    df_dates_filled.loc[df_dates_filled['Revision_Date'] < df_dates_filled['Release_Date'], 'Release_Date']
### Dataframe types control:
print(df_dates_filled.dtypes)
### Filled dates saving:
df_dates_filled.to_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled, mode = 'a')

In [ ]:
### RUN TO TESTING: CHECKING FOR FINAL VALUE WITHOUT FINAL DATE:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
df_dates_filled.loc[df_dates_filled['Final_Value'].notna() & df_dates_filled['Final_Date'].isna()]

In [ ]:
### RUN TO TESTING: CHECKING FOR REVISION DATE TO BE AFTER RELEASE DATE:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
df_dates_filled.loc[df_dates_filled['Revision_Date'] < df_dates_filled['Release_Date']]

In [ ]:
### RUN TO TESTING: CHECKING FOR FINAL DATE TO BE AFTER RELEASE DATE:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
df_dates_filled.loc[df_dates_filled['Final_Date'] < df_dates_filled['Release_Date']]

In [ ]:
### RUN TO TESTING: CHECKING FOR FINAL DATE TO BE AFTER REVISION DATE:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
df_dates_filled.loc[df_dates_filled['Final_Date'] < df_dates_filled['Revision_Date']]

In [ ]:
### RUN TO TESTING: CHECKING FOR RELEASE PAIR COMPLETENESS:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
print(df_dates_filled.loc[df_dates_filled['Release_Date'].isna() & df_dates_filled['Release_Value'].notna()])
print(df_dates_filled.loc[df_dates_filled['Release_Date'].notna() & df_dates_filled['Release_Value'].isna()])

In [ ]:
### RUN TO TESTING: CHECKING FOR REVISION PAIR COMPLETENESS:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
print(df_dates_filled.loc[df_dates_filled['Revision_Date'].isna() & df_dates_filled['Revision_Value'].notna()])
print(df_dates_filled.loc[df_dates_filled['Revision_Date'].notna() & df_dates_filled['Revision_Value'].isna()])

In [ ]:
### RUN TO TESTING: CHECKING FOR FINAL PAIR COMPLETENESS:

df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
print(df_dates_filled.loc[df_dates_filled['Final_Date'].isna() & df_dates_filled['Final_Value'].notna()])
print(df_dates_filled.loc[df_dates_filled['Final_Date'].notna() & df_dates_filled['Final_Value'].isna()])

In [ ]:
### RUN TO TESTING: FILLED ECO INDICES DATA PLOTTING TO VISUAL CONTROL

### Index choosing:
str_test_index = 'ADP CHNG Index'
### Data loading:
df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
df_flags = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_flags)
### Flags printing:
print(df_flags.loc[str_test_index, All])
### Series creating:
df_eco_ind = df_dates_filled.loc[(str_test_index, All), All].droplevel(0)[['Release_Value', 'Revision_Value', 'Final_Value']]
date_xlim_start = df_eco_ind.index.min()
date_xlim_finish = df_eco_ind.index.max()
#date_xlim_start = datetime(1984, 12, 31)
date_xlim_finish = datetime(2020, 1, 31)
flo_ylim_min = df_eco_ind.loc[date_xlim_start : date_xlim_finish, All].min().min()
flo_ylim_max = df_eco_ind.loc[date_xlim_start : date_xlim_finish, All].max().max()
tup_ylim = (flo_ylim_min - abs(flo_ylim_min) / 10, flo_ylim_max + abs(flo_ylim_max) / 10)
ax_eco_ind = df_eco_ind.plot(figsize = (20, 7.5), title = str_test_index + ' revisions', 
                style = ['c+', 'mx', 'k-'], markersize = 7.5, markeredgewidth = 1.5, 
                xlim = (date_xlim_start, date_xlim_finish), ylim = tup_ylim, x_compat = True)
ax_eco_ind.xaxis.set_major_locator(mdates.YearLocator(base = 1))                             
ax_eco_ind.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))                
plt.show() 

In [ ]:
### RUN TO RE-EXPORT DATA: VALUES STACKING

### Dataframe loading:
df_dates_filled = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_filled)
### Release pair stacking:
df_release_pair = df_dates_filled[['Release_Date', 'Release_Value']]
df_release_pair.columns = ['Observation_Date', 'Index_Value']
df_release_pair = df_release_pair.dropna()
ser_release_based = df_release_pair.set_index('Observation_Date', append = True).squeeze()
#df_release_pair['Stage'] = 'Release'
#ser_release_based = df_release_pair.set_index(['Observation_Date', 'Stage'], append = True).squeeze()
### First revision pair stacking:
df_revision_pair = df_dates_filled[['Revision_Date', 'Revision_Value']]
df_revision_pair.columns = ['Observation_Date', 'Index_Value']
df_revision_pair = df_revision_pair.dropna()
ser_revision_based = df_revision_pair.set_index('Observation_Date', append = True).squeeze()
#df_revision_pair['Stage'] = 'First Revision'
#ser_revision_based = df_revision_pair.set_index(['Observation_Date', 'Stage'], append = True).squeeze()
### Final revision pair stacking:
df_final_pair = df_dates_filled[['Final_Date', 'Final_Value']]
df_final_pair.columns = ['Observation_Date', 'Index_Value']
df_final_pair = df_final_pair.dropna()
ser_final_based = df_final_pair.set_index('Observation_Date', append = True).squeeze()
#df_final_pair['Stage'] = 'Final Revision'
#ser_final_based = df_final_pair.set_index(['Observation_Date', 'Stage'], append = True).squeeze()
### Vectors aggregating:
ser_history_raw = ser_final_based.combine_first(ser_revision_based).combine_first(ser_release_based).dropna().sort_index()

In [ ]:
### RUN TO RE-EXPORT DATA: FREQUENCY RECREATING

### 77 : BAKETOT Index frequency rerceating:
ser_history_raw.loc['BAKETOT Index', pd.to_datetime('1994-12-23'), pd.to_datetime('1994-12-23')] = np.NaN
ser_history_raw.loc['BAKETOT Index', pd.to_datetime('1995-12-22'), pd.to_datetime('1995-12-22')] = np.NaN
### 78 : IEIPIMOM Index frequency rerceating:
ser_history_raw.loc['IEIPIMOM Index', pd.to_datetime('2004-08-31'), pd.to_datetime('2004-08-31')] = np.NaN
### 79 : IERSVMOM Index frequency rerceating:
ser_history_raw.loc['IERSVMOM Index', pd.to_datetime('2001-01-31'), pd.to_datetime('2001-01-31')] = np.NaN
ser_history_raw.loc['IERSVMOM Index', pd.to_datetime('2001-02-28'), pd.to_datetime('2001-02-28')] = np.NaN
### 80 : ITBCI Index frequency rerceating:
ser_history_raw.loc['ITBCI Index', pd.to_datetime('2020-04-30'), pd.to_datetime('2020-04-30')] = np.NaN
### 81 : ITPSSA Index frequency rerceating:
ser_history_raw.loc['ITPSSA Index', pd.to_datetime('2020-04-30'), pd.to_datetime('2020-04-30')] = np.NaN
### 82 : JCOMHCF Index frequency rerceating:
idx_ticker_date = ser_history_raw.loc[['JCOMHCF Index'], All, All].index.get_level_values('Data_Date').unique()
idx_ticker_date = pd.date_range(start = '1984-12-31', end = '2004-03-31', freq = 'M').difference(idx_ticker_date)
idx_ticker_reindex = pd.MultiIndex.from_tuples(tuple(zip(['JCOMHCF Index'] * len(idx_ticker_date), idx_ticker_date, idx_ticker_date)))
ser_ei_to_add = pd.Series(np.NaN, index = idx_ticker_reindex)
ser_ei_to_add.index.names = ser_history_raw.index.names
ser_history_raw = pd.concat([ser_history_raw, ser_ei_to_add], axis = 0)
### 83 : JNPIY Index frequency rerceating:
ser_history_raw.loc['JNPIY Index', pd.to_datetime('2001-02-28'), pd.to_datetime('2001-02-28')] = np.NaN
### 84 : MAPMINDX Index frequency rerceating:
ser_history_raw.loc['MAPMINDX Index', pd.to_datetime('2016-05-31'), pd.to_datetime('2016-05-31')] = np.NaN
ser_history_raw.loc['MAPMINDX Index', pd.to_datetime('2016-06-30'), pd.to_datetime('2016-06-30')] = np.NaN
ser_history_raw.loc['MAPMINDX Index', pd.to_datetime('2016-07-31'), pd.to_datetime('2016-07-31')] = np.NaN
ser_history_raw.loc['MAPMINDX Index', pd.to_datetime('2016-08-31'), pd.to_datetime('2016-08-31')] = np.NaN
ser_history_raw.loc['MAPMINDX Index', pd.to_datetime('2016-09-30'), pd.to_datetime('2016-09-30')] = np.NaN
### 85 : MPMIGBMA Index frequency rerceating:
idx_ticker_date = pd.date_range(start = '2014-01-31', end = '2015-11-30', freq = 'M')
idx_ticker_reindex = pd.MultiIndex.from_tuples(tuple(zip(['MPMIGBMA Index'] * len(idx_ticker_date), idx_ticker_date, idx_ticker_date)))
ser_ei_to_add = pd.Series(np.NaN, index = idx_ticker_reindex)
ser_ei_to_add.index.names = ser_history_raw.index.names
ser_history_raw = pd.concat([ser_history_raw, ser_ei_to_add], axis = 0)
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2012-05-31'), pd.to_datetime('2012-05-31')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2013-02-28'), pd.to_datetime('2013-02-28')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2016-01-31'), pd.to_datetime('2016-01-31')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2016-02-29'), pd.to_datetime('2016-02-29')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2016-06-30'), pd.to_datetime('2016-06-30')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2016-07-31'), pd.to_datetime('2016-07-31')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2016-08-31'), pd.to_datetime('2016-08-31')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2016-12-31'), pd.to_datetime('2016-12-31')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2017-03-31'), pd.to_datetime('2017-03-31')] = np.NaN
ser_history_raw.loc['MPMIGBMA Index', pd.to_datetime('2017-04-30'), pd.to_datetime('2017-04-30')] = np.NaN
### 86 : NEISIYOY Index frequency rerceating:
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('1999-07-31'), pd.to_datetime('1999-07-31')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('1999-08-31'), pd.to_datetime('1999-08-31')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2000-06-30'), pd.to_datetime('2000-06-30')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2000-07-31'), pd.to_datetime('2000-07-31')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2000-09-30'), pd.to_datetime('2000-09-30')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2000-11-30'), pd.to_datetime('2000-11-30')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2001-07-31'), pd.to_datetime('2001-07-31')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2002-07-31'), pd.to_datetime('2002-07-31')] = np.NaN
ser_history_raw.loc['NEISIYOY Index', pd.to_datetime('2003-07-31'), pd.to_datetime('2003-07-31')] = np.NaN
### 87 : NERS20Y Index frequency rerceating:
ser_history_raw.loc['NERS20Y Index', pd.to_datetime('2001-07-31'), pd.to_datetime('2001-07-31')] = np.NaN
### 88 : PTCCI Index frequency rerceating:
ser_history_raw.loc['PTCCI Index', pd.to_datetime('2004-05-31'), pd.to_datetime('2004-05-31')] = np.NaN
### 89 : PTCPHAMM Index frequency rerceating:
ser_history_raw.loc['PTCPHAMM Index', pd.to_datetime('2019-01-31'), pd.to_datetime('2019-01-31')] = np.NaN
### 90 : UKCCI Index frequency rerceating:
ser_history_raw.loc['UKCCI Index', pd.to_datetime('1995-05-31'), pd.to_datetime('1995-05-31')] = np.NaN
### Index sorting:
ser_history_raw = ser_history_raw.sort_index()
### Results saving:
ser_history_raw.to_hdf(str_path_bb_idx_hdf, key = str_key_raw_history, mode = 'a')

In [ ]:
### RUN TO TESTING: FREQUENCY CONTROL

ser_history_raw = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_history)
ser_idx_test = ser_history_raw.to_frame().reset_index('Data_Date')['Data_Date'].squeeze().reset_index('Observation_Date', drop = True)
ser_idx_test = ser_idx_test.groupby('Index_Name', group_keys = False).apply(lambda ser_ticker: ser_ticker.drop_duplicates())
df_idx_test = ser_idx_test.groupby('Index_Name').apply(lambda ser_ticker: (ser_ticker - ser_ticker.shift()).dt.days).groupby('Index_Name').agg(['min', 'max', 'median'])
df_idx_test['delta'] = df_idx_test['max'] - df_idx_test['min']
df_idx_test.sort_values('delta')

In [ ]:
### RUN TO RE-EXPORT DATA: HISTORY REINDEXATION TO OBSERVATION DATE BUSINESS DAILY MATRIX STRUCTURE (PRELIMINARY ACTIONS)

### Data loading:
ser_history_raw = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_raw_history)
### Moving all dates to tne nearest Business day:
df_history_raw = ser_history_raw.to_frame().reset_index('Observation_Date')
df_history_raw['Observation_Date'] = df_history_raw['Observation_Date'] + 0 * pd.offsets.BDay()
ser_history_bday = df_history_raw.set_index('Observation_Date', append = True).squeeze()
ser_history_bday = ser_history_bday.loc[All, All, idx_date_range]
### Data adding to hdf collection:
ser_history_bday.to_hdf(str_path_bb_idx_hdf, key = str_key_bday_history, mode = 'a') ### BDay moved observation dates

In [ ]:
### RUN TO RE-EXPORT DATA: HISTORY DATA DIAGONAL EXTRACTION

def get_diagonal(ser_name, idx_date_range):
    ### Defining triangle extraction:
    def triangle_filter(ser_date):
        date_diag = ser_date.index.get_level_values('Data_Date')[0]
        ser_result = ser_date.droplevel('Data_Date')
        ser_result = ser_result[ser_result.index >= date_diag] 
        return ser_result    
    ### EI name extracting:
    str_index_name = ser_name.index.get_level_values(0)[0]
    ### Observation dates reindexation:    
    print(ser_name.index.get_level_values(0)[0], ': Reindexation')    
    idx_observation_range = ser_name.index.get_level_values('Observation_Date').dropna().unique().intersection(idx_date_range)
    ser_full = ser_name.droplevel('Index_Name').unstack('Data_Date').reindex(idx_observation_range).stack('Data_Date', dropna = False).squeeze()
    ser_full = ser_full.swaplevel()
    ser_full.index.rename('Observation_Date', level = -1, inplace = True)    
    ### Forward filling for each data date:
    ser_full = ser_full.groupby('Data_Date').ffill()   
    ### Diagonalization:
    ser_triangle = ser_full.groupby('Data_Date').apply(triangle_filter).sort_index()
    ### Creating future diagonal vector:
    ser_iter_diag = pd.Series(np.NaN, idx_observation_range)
    ### Looping over unique dates:
    for iter_date in idx_observation_range:
        ### Trying to get the latest data date observation:
        try:
            ser_iter_diag[iter_date] = ser_name.droplevel('Index_Name').loc[All, iter_date].dropna()[-1]
        except:
            pass
    ser_iter_diag = ser_iter_diag.reindex(idx_date_range).ffill()
    ### Results output:
    return ser_iter_diag
### Economic Indices vector loading:
ser_history_bday = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_bday_history)
### Data transforming:
ser_diagonal_raw = ser_history_bday.groupby('Index_Name').apply(get_diagonal, idx_date_range)    
### Saving results to hdf file:
ser_diagonal_raw.to_hdf(str_path_bb_diag_hdf, key = str_key_diag_daily_raw, mode = 'w')

In [42]:
### RUN TO RE-EXPORT DATA: GROUPING RESEARCH TO SELECT BASIS TICKERS

### Defining minimum Data Date for each ticker:
ser_history_bday = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_bday_history)
ser_min_date = ser_history_bday.to_frame().reset_index('Data_Date').drop(0, axis = 1).reset_index('Observation_Date', drop = True).squeeze().groupby('Index_Name').agg(min)
ser_min_date.name = 'Min_Date'
### Creating description table for all types combinations:
df_flags = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_flags)
### Defining description table generator:
def type_analyzer(df_type):
    ### Type tickers distribution by region calculating:
    ser_region_distribution = df_type.groupby('Region').apply(lambda df_region: len(df_region.index))
    ser_region_distribution['All'] = len(df_type.index)
    ser_region_distribution = pd.concat([ser_region_distribution], keys = ['Regional_Presence'], names = ['Info'])
    ### Adding information about basis ticker for each type:
    ser_region_distribution['Oldest_Ticker_Info', 'Oldest_Name'] = df_type['Min_Date'].idxmin()
    ser_region_distribution['Oldest_Ticker_Info', 'Region'] = df_type.loc[df_type['Min_Date'].idxmin(), 'Region']
    ser_region_distribution['Oldest_Ticker_Info', 'Min_Date'] = df_type.loc[df_type['Min_Date'].idxmin(), 'Min_Date']    
    ser_region_distribution['Oldest_Ticker_Info', 'Description'] = df_type.loc[df_type['Min_Date'].idxmin(), 'Description']     
    ### Results output:
    return ser_region_distribution
### Extending flags table and dropping tickers, that were cleared earlier:
df_flags_typed = pd.concat([df_flags, ser_min_date], axis = 1, sort = False)
df_flags_typed = df_flags_typed.loc[df_flags_typed['Min_Date'].notna()]
### Description table generating:
df_type_info = df_flags_typed.groupby(['Type_Prime', 'Type_Second']).apply(type_analyzer)
df_type_info = df_type_info.unstack(['Info', 'Region'], fill_value = 0)
df_type_info.columns.names = ['', '']
### Binding each ticker with it's basis:
df_oldest_ticker = df_type_info[[('Oldest_Ticker_Info', 'Oldest_Name'), ('Oldest_Ticker_Info', 'Min_Date')]]
df_oldest_ticker.columns = ['Basic_Ticker', 'Basic_Min_Date']
df_flags_typed = df_flags_typed.join(df_oldest_ticker, how = 'left', on = ['Type_Prime', 'Type_Second'])
### Clearing basis ticker for basis tickers and resort values to guarantee basis tickers priority:
df_flags_typed.loc[df_flags_typed.index == df_flags_typed['Basic_Ticker'], 'Basic_Ticker'] = ''
df_flags_typed = df_flags_typed.groupby('Basic_Ticker', group_keys = False).apply(lambda df_group: df_group.sort_index())
### Adding flag to define if we need to rebase ticker:
df_flags_typed['Rebase_Flag'] = True
df_flags_typed.loc[df_flags_typed['Basic_Ticker'] == '', 'Rebase_Flag'] = False
df_flags_typed.loc[df_flags_typed['Min_Date'] < (datetime_basis - pd.DateOffset(years = int_not_to_rebase_term)), 'Rebase_Flag'] = False
df_flags_typed.loc[df_flags_typed['Min_Date'] < (df_flags_typed['Basic_Min_Date'] - pd.DateOffset(years = int_not_to_rebase_diff)), 'Rebase_Flag'] = False
df_flags_typed.loc[df_flags_typed['Basic_Min_Date'] > datetime_basis, 'Rebase_Flag'] = False
#df_type_info.to_excel('Data_Files/Test_Files/Ticker_Info.xlsx', merge_cells = True)
df_flags_typed.to_excel('Data_Files/Test_Files/Ticker_Info.xlsx', merge_cells = False)

In [20]:
### RUN TO RE-EXPORT DATA: HISTORY DATA TRANSFORMATION (REINDEXATION, FLAGS IMPLEMENTATION, Z-SCORING)

### Defining Economic Index series transformation:
def complex_transform(ser_name, idx_date_range, df_flags, int_max_name_length, int_min_years, bool_perform_sa = False):
    ### Local constants definition:
    dict_interval = {}
    dict_interval['Q'] = 4
    dict_interval['M'] = 12
    dict_interval['W'] = 52      
    ### Defining triangle extraction:
    def triangle_filter(ser_date):
        ### Extracting particular Data Date:
        date_diag = ser_date.index.get_level_values('Data_Date')[0]
        ### Dropping constant level:
        ser_result = ser_date.droplevel('Data_Date')
        ### Filtering over-diagonal values:
        ser_result = ser_result[ser_result.index >= date_diag] 
        ### Results output:
        return ser_result
    ### Year-over-year-percent ticker values transforming to stock-like series:
    def yoy_to_level(ser_date, int_step):
        ### Dropping constant level:
        ser_result = ser_date.droplevel('Observation_Date')
        ### Basis initiating:
        flo_basement = 1.0
        ### Factor initiating: 
        flo_next_brick  = 1.0
        ### Looping over month numbers:
        for iter_period in range(min(int_step, len(ser_result.index))):         
            ### Basement building up:
            flo_basement = flo_basement * flo_next_brick
            ### Next basement brick producing:
            flo_next_brick = ((flo_next_brick ** (iter_period)) * (ser_result.iloc[iter_period] ** (1 / int_step))) ** (1 / (iter_period + 1)) 
            ### Jumping cumulative product performing:
            idx_iter_data = ser_result.index[iter_period :: int_step]
            ser_result.loc[idx_iter_data] = ser_result.loc[idx_iter_data].cumprod() * flo_basement       
        ### Results output:            
        return ser_result    
    ### X13 ARIMA Seasonality adjustment model:
    def perform_x13_sa(ser_date):
        ### Dropping constant level:        
        ser_result = ser_date.droplevel('Observation_Date')
        ### Check for minimal quantity of observations to z-score:
        if (ser_result.last_valid_index() - ser_result.first_valid_index()).days >= (int_min_years * 365):   
            ### Naming series for x13 performing:
            ser_result.name = 'Ticker'
            ### Calculating shift value to make all series positive:
            flo_positron = abs(ser_result.min()) * 2
            try:
                ### Performing seasonality adjustment:
                ser_result = x13_arima_analysis(ser_result + flo_positron, outlier = True, trading = True).seasadj - flo_positron
                print('SA success : ', ser_date.index.get_level_values('Observation_Date')[0])                 
            except:
                print('SA error : ', ser_date.index.get_level_values('Observation_Date')[0]) 
        ### Results output:                
        return ser_result 
    ### Extracting Observation Date column for ticker:
    def get_obs_date_vector(str_ticker, str_date, bool_exact_date = False, bool_drop_levels = False):
        ### Vector for exact date:
        if bool_exact_date:
            ser_obs_date = pd.read_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z, where = 'Index_Name == str_ticker & Observation_Date == str_date')
        ### Vector for nearest date:        
        else:
            ### Loading full ticker series:        
            ser_z_scored = pd.read_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z, where = 'Index_Name == str_ticker & Observation_Date <= str_date')
            ### Extracting data for max date less or equal to needed date:
            ser_obs_date = ser_z_scored.loc[All, All, [ser_z_scored.index.levels[-1].max()]]
        ### Dropping constant index levels if needed:
        if bool_drop_levels:
            return ser_obs_date.droplevel(['Index_Name', 'Observation_Date'])
        else:
            return ser_obs_date    
    ### Defining time-vector z-scoring procedure:    
    def by_date_z_score(ser_date, int_winsorize_bound, flo_winsorize_tolerance, int_winsorize_steps_limit, int_min_years, str_basis_index):
        ### Check for empty vector (doing nothing):
        if ser_date.count():
            ### Check for non-constant vector:
            if (ser_date.std() > flo_winsorize_tolerance):
                ### Check for minimal quantity of observations to z-score:
                if (ser_date.last_valid_index()[0] - ser_date.first_valid_index()[0]).days >= (int_min_years * 365):   
                    ### Calculating of z scores:
                    ser_date = (ser_date - ser_date.mean()) / ser_date.std()        
                    bool_to_winsor = True   
                    int_iter = 1
                    while (bool_to_winsor): 
                        int_iter += 1                
                        ### Value based winsorization:                
                        ser_date.clip(lower = -int_winsorize_bound, upper = int_winsorize_bound, inplace = True)
                        ### Recalculating of z scores:
                        ser_date = (ser_date - ser_date.mean()) / ser_date.std()
                        ### Checking for boundaries and steps:
                        if((ser_date.loc[ser_date.abs() >= (int_winsorize_bound + flo_winsorize_tolerance)].count() == 0) | (int_iter > int_winsorize_steps_limit)):
                            bool_to_winsor = False
                    ### Checking for basis index:
                    if str_basis_index:
                        ### Extracting column from z-scored basis ticker series:
                        str_obs_date = ser_date.index[0][1].strftime('%Y-%m-%d')
                        ser_basis_date = get_obs_date_vector(str_basis_index, str_obs_date, bool_exact_date = False, bool_drop_levels = True)
                        ser_date = ser_date * ser_basis_date.std() + ser_basis_date.mean()
                else:
                    ### Killing values that we can't z-score
                    ser_date.loc[All] = np.NaN
            else:
                ### Check for minimal quantity of observations to z-score:
                if (ser_date.last_valid_index()[0] - ser_date.first_valid_index()[0]).days >= (int_min_years * 365):             
                    ### Constant values demeaning:
                    ser_date = ser_date - ser_date.mean()
                    ### Checking for basis index:
                    if str_basis_index:
                        ### Extacting column from z-scored basis ticker series:
                        str_obs_date = ser_date.index[0][1].strftime('%Y-%m-%d')
                        ser_basis_date = get_obs_date_vector(str_basis_index, str_obs_date, bool_exact_date = False, bool_drop_levels = True)
                        ser_date = ser_date * ser_basis_date.std() + ser_basis_date.mean()                    
                else:
                    ### Killing values that we can't z-score
                    ser_date.loc[All] = np.NaN
        ### Memory optimization:
        ser_date = ser_date.astype('float32')
        return ser_date    
    ### EI name extracting:
    str_index_name = ser_name.index.get_level_values(0)[0]
    ### Observation dates reindexation:    
    print(ser_name.index.get_level_values(0)[0], ': Reindexation')    
    idx_observation_range = ser_name.index.get_level_values('Observation_Date').unique().intersection(idx_date_range).sort_values()
    ser_full = ser_name.droplevel('Index_Name').unstack('Data_Date').reindex(idx_observation_range).stack('Data_Date', dropna = False).squeeze()
    ser_full = ser_full.swaplevel()
    ser_full.index.rename('Observation_Date', level = -1, inplace = True)    
    ### Forward filling for each data date:
    ser_full = ser_full.groupby('Data_Date').ffill()   
    ### Diagonalization:
    ser_triangle = ser_full.groupby('Data_Date').apply(triangle_filter).sort_index()
    ### Flags extracting:
    ser_flags = df_flags.loc[str_index_name, All].squeeze() 
    ### 'TAR' type checking:
    if (ser_flags['Type_Prime'] == 'TAR'):
        print(str_index_name, ': TAR Primary Type ignoring')        
        pass
    ### Flags-based transforming:
    else:
        ### Indices of NA values collecting:
        idx_isna = ser_triangle.loc[ser_triangle.isna()].index
        ### Transforming to stock-like series:
        if (ser_flags['Processing'] in ['Index', 'Level', 'Level%']):
            ser_stock = ser_triangle
        elif (ser_flags['Processing'] == 'Flow'):
            print(str_index_name, ': Transformation to stock-like series: Cumulative sum')
            ### Filling empty values:
            ser_triangle = ser_triangle.fillna(0)
            ### Cumulative sum for each observation date calculating:
            ser_stock = ser_triangle.groupby('Observation_Date').cumsum()
            ### Dropping NA values:
            ser_stock.loc[idx_isna] = np.NaN
        else:
            print(str_index_name, ': Transformation to stock-like series: Cumulative product')
            ### Filling empty values:
            ser_triangle = ser_triangle.fillna(0)
            ### Percents to multipliers converting:
            ser_stock = 1 + ser_triangle / 100
            ### Calculating with needed periodicity:
            if (ser_flags['Frequency'] == 'Monthly'):
                int_step = dict_cumprod_step[ser_flags['Processing']]
                ### Year-by-year cumprod with rebasing:
                ser_stock = ser_stock.groupby('Observation_Date').apply(yoy_to_level, int_step).swaplevel().sort_index()
            ### Dropping NA values:
            ser_stock.loc[idx_isna] = np.NaN
        ### Seasonality adjustment testing:
        if bool_perform_sa:
            print(str_index_name, ': Seasonality adjustment')            
            ### Filling empty values:            
            ser_stock = ser_stock.groupby('Observation_Date').ffill()
            ser_stock = ser_stock.groupby('Observation_Date').apply(perform_x13_sa).swaplevel().sort_index()
            ### Dropping NA values:
            ser_stock.loc[idx_isna] = np.NaN                        
        ### Transforming to YoY series:
        if (ser_flags['Processing'] == 'Index'):
            ### Debasing only:
            print(str_index_name, ': Transformation to YoY series: Debasing')            
            ser_yoy = ser_stock - ser_flags['Base']
        elif (ser_flags['Processing'] in ['Flow', 'Level']):    
            ### Simple difference:
            print(str_index_name, ': Transformation to YoY series: Simple difference')
            ### Shifting lag defining:
            if (ser_flags['Frequency'] in dict_yoy_shift.keys()):
                int_yoy_shift = dict_yoy_shift[ser_flags['Frequency']]
            else:
                int_yoy_shift = dict_yoy_shift['Other']            
            ### Stock-like series differing:
            ser_yoy = ser_stock.groupby('Observation_Date', group_keys = False).apply(lambda ser_obs_date: ser_obs_date - ser_obs_date.shift(int_yoy_shift))
        else:      
            ### Difference with dividing:
            print(str_index_name, ': Transformation to YoY series: Difference with dividing')
            ### Shifting lag defining:
            if (ser_flags['Frequency'] in dict_yoy_shift.keys()):
                int_yoy_shift = dict_yoy_shift[ser_flags['Frequency']]
            else:
                int_yoy_shift = dict_yoy_shift['Other']
            ### Stock-like series differing:
            ser_yoy = ser_stock.groupby('Observation_Date', group_keys = False)\
                               .apply(lambda ser_obs_date: (ser_obs_date / ser_obs_date.shift(int_yoy_shift) - 1))  
        ser_yoy.name = 'YoY'
        ### Z-scoring across the observation dates:
        print(ser_name.index.get_level_values(0)[0], ': Z-scoring across the observation dates')
        ser_yoy_z = ser_yoy.groupby('Observation_Date').apply(by_date_z_score, int_winsorize_bound, flo_winsorize_tolerance, int_winsorize_steps_limit, int_min_years,
                                                              ser_flags['Basic_Ticker'])
        ### Adding results to matrix cube:
        pd.concat([ser_yoy_z], keys = [str_index_name], names = ['Index_Name']).to_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z, format = 'table',
                                                                                       complevel = 9, append = True, mode = 'a',
                                                                                       min_itemsize = {'Index_Name': int_max_name_length})    
#    ### Results output:
#    return pd.concat([ser_yoy_z], keys = [str_index_name], names = ['Index_Name'])

### Economic Indices vector loading:
ser_history_bday = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_bday_history)
ser_history_bday = ser_history_bday.reindex(df_flags_typed.index, level = 'Index_Name')
### Testing:
if os.path.isfile(str_path_bb_matrix_hdf):
    os.remove(str_path_bb_matrix_hdf)
ser_history_bday_test = ser_history_bday.loc[ser_history_bday.index.get_level_values('Index_Name').unique()[[0, 21]], All, All]
### Maximum length calculating (for HDF manipulations):
int_max_name_length = max(ser_history_bday.index.levels[0].str.len())
### Data transforming:
ser_history_bday_test.groupby('Index_Name').apply(complex_transform, idx_date_range, df_flags_typed, int_max_name_length, int_min_years)
#set_res_test = ser_history_bday_test.groupby('Index_Name', group_keys = False).apply(complex_transform, idx_date_range, df_flags, int_max_name_length, int_min_years)

BEBCI Index : Reindexation
BEBCI Index : Transformation to YoY series: Debasing
BEBCI Index : Z-scoring across the observation dates
EUSCEMU Index : Reindexation
EUSCEMU Index : Transformation to YoY series: Debasing
EUSCEMU Index : Z-scoring across the observation dates


Index_Name
BEBCI Index      None
EUSCEMU Index    None
Name: 0, dtype: object

In [ ]:
### TEMP: RESULTS CHECK

#ser_history_bday_test.to_excel('Data_Files/Test_Files/Processing_test.xlsx', merge_cells = False)
#set_res_test.to_excel('Data_Files/Test_Files/Processing_test.xlsx', merge_cells = False)
#import tables
#tables.file._open_files.close_all()
#pd.read_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z).to_excel('Data_Files/Test_Files/Processing_test.xlsx', merge_cells = False)
ser_history_bday.loc['UKMSVTVJ Index', All, All]

In [ ]:
### DEFINING OBSERVATION DATE VECTOR EXTRACTION

def get_obs_date_vector(str_ticker, str_date, bool_exact_date = False, bool_drop_levels = False):
    ### Vector for exact date:
    if bool_exact_date:
        ser_obs_date = pd.read_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z, where = 'Index_Name == str_ticker & Observation_Date == str_date')
    ### Vector for nearest date:        
    else:
        ### Loading full ticker series:        
        ser_z_scored = pd.read_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z, where = 'Index_Name == str_ticker & Observation_Date <= str_date')
        ### Extracting data for max date less or equal to needed date:
        ser_obs_date = ser_z_scored.loc[All, All, [ser_z_scored.index.levels[-1].max()]]
    ### Dropping constant index levels if needed:
    if bool_drop_levels:
        return ser_obs_date.droplevel(['Index_Name', 'Observation_Date'])
    else:
        return ser_obs_date

In [ ]:
### RUN TO RE-EXPORT DATA: Z_SCORED EI DIAGONAL CONSTRUCTING:

### Creating container for tickers diagonals:
dict_ei_diag = {}
### Economic Indices vector loading:
ser_history_bday = pd.read_hdf(str_path_bb_idx_hdf, key = str_key_bday_history)
### Tickers list preparing:
idx_ticker_list = ser_history_bday.index.levels[0]
### Looping over tickers:
for iter_ticker in idx_ticker_list:
    print(iter_ticker)
    ### Loading matrix for each ticker:
    ser_iter_matrix = pd.read_hdf(str_path_bb_matrix_hdf, key = str_key_matrix_z, where = 'Index_Name == iter_ticker').droplevel('Index_Name')
    ### Extracting unique observation dates for each ticker later than diagonal start date:
    idx_date_list = ser_iter_matrix.loc[All, date_diag_start : ].index.dropna().get_level_values('Observation_Date').unique()
    ### Creating future diagonal vector:
    ser_iter_diag = pd.Series(np.NaN, idx_date_list)    
    ### Determining first valid date and first date to place z-scoring results on diagonal:
    ### Looping over unique dates:
    for iter_date in idx_date_list:
        ### Trying to get the latest data date observation:
        try:
            ser_iter_diag[iter_date] = ser_iter_matrix.loc[All, iter_date].dropna()[-1]
        except:
            pass
    ### Checking for data earlier than diagonal start date:
    if ((ser_iter_matrix.index.get_level_values('Observation_Date').unique().min() < date_diag_start) & pd.notna(ser_iter_diag.values[0])):
        ### Turning diagonal start date column to diagonal with data dates as index dates:
        date_before_diag = ser_iter_matrix.index.levels[-1][list(ser_iter_matrix.index.levels[-1]).index(idx_date_list[0]) - 1]
        ### Ticker values on the date to turn:
        list_iter_values_to_turn = ser_iter_matrix.loc[ : idx_date_list[0] - pd.offsets.Day(1), [idx_date_list[0]]].values
        ### First appearance dates extracting:
        ser_iter_release = ser_iter_matrix.loc[ : idx_date_list[0] - pd.offsets.Day(1), :].groupby('Data_Date')\
                                                         .apply(lambda ser_date: ser_date.droplevel('Data_Date').first_valid_index())
        ### Implementing lag before the announcement for the first valid observation date (3 years of data dates with equal first valid index):
        date_second_obs = pd.to_datetime(ser_iter_release.loc[ser_iter_release > ser_iter_release.min()].values[0])
        ins_second_obs_lag = (date_second_obs - ser_iter_release.loc[ser_iter_release > ser_iter_release.min()].index[0]).days
        ser_iter_release.loc[ser_iter_release < date_second_obs] = ser_iter_release.loc[ser_iter_release < date_second_obs].index + pd.offsets.Day(ins_second_obs_lag)
        ### Taking announcement dates as an index for firsat column values:
        list_iter_index_to_turn = ser_iter_release.values
        ### Modified column to turn:
        ser_iter_to_turn = pd.Series(list_iter_values_to_turn, index = list_iter_index_to_turn)
        ### Dropping repeated dates and empty dates:
        ser_iter_to_turn = ser_iter_to_turn.groupby(level = 0).apply(lambda ser_date: ser_date[-1]).dropna()
        ### Cutting series not to intersect with diagonal index:
        ser_iter_to_turn = ser_iter_to_turn.loc[ : idx_date_list[0] - pd.offsets.Day(1)]
        ### Joining series:
        ser_iter_diag = pd.concat([ser_iter_to_turn, ser_iter_diag], axis = 0).sort_index() 
    ### Reindexation to business daily vector and forward filling:
    ser_iter_diag = ser_iter_diag.ffill().reindex(idx_date_range).ffill()
    ### Saving ticker diagonal to the container:
    dict_ei_diag[iter_ticker] = ser_iter_diag
### Aggregating ticker diagonals:
ser_diagonal_z = pd.concat(dict_ei_diag, axis = 0)
ser_diagonal_z.index.names = ['Index_Name', 'Date']
ser_diagonal_z.name = 'EI_diagonal'
### Saving results to hdf file:
ser_diagonal_z.to_hdf(str_path_bb_diag_hdf, key = str_key_diag_daily_z, mode = 'a')

In [ ]:
### RUN TO TESTING: DIAGONALS PLOTTING

str_test_ticker = 'DGNOCHNG Index'
ser_diagonal = pd.read_hdf(str_path_bb_diag_hdf, key = str_key_diag_daily_z)
#ser_diagonal = pd.read_hdf(str_path_bb_diag_hdf, key = str_key_diag_daily_raw)
ser_eco_ind = ser_diagonal.loc[str_test_ticker, All].droplevel(0).dropna()
date_xlim_start = ser_eco_ind.index.min()
date_xlim_finish = ser_eco_ind.index.max()
flo_ylim_min = ser_eco_ind.loc[date_xlim_start : date_xlim_finish].min()
flo_ylim_max = ser_eco_ind.loc[date_xlim_start : date_xlim_finish].max()
tup_ylim = (flo_ylim_min - abs(flo_ylim_min) / 10, flo_ylim_max + abs(flo_ylim_max) / 10)
ax_eco_ind = ser_eco_ind.plot(figsize = (20, 7.5), title = str_test_ticker + ' diagonal', xlim = (date_xlim_start, date_xlim_finish), ylim = tup_ylim, x_compat = True)
ax_eco_ind.xaxis.set_major_locator(mdates.YearLocator(base = 1))                             
ax_eco_ind.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))                
plt.show() 

In [ ]:
### RUN TO TESTING: DIAGONALS EXPORT TO MS EXCEL

ser_diagonal_z = pd.read_hdf(str_path_bb_diag_hdf, key = str_key_diag_daily_z)
ser_diagonal_z.unstack('Index_Name').to_excel('Data_Files/Test_Files/Diag_Eco_Indices_z.xlsx', merge_cells = False)
ser_diagonal_raw = pd.read_hdf(str_path_bb_diag_hdf, key = str_key_diag_daily_raw)
ser_diagonal_raw.unstack('Index_Name').to_excel('Data_Files/Test_Files/Diag_Eco_Indices_raw.xlsx', merge_cells = False)